<a href="https://colab.research.google.com/github/dataplusplus-ai/tensorflow_Datapipeline/blob/main/feature_extractor_from_mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
import tensorflow_hub as hub
model_selection = ("mobilenet_v2", 224, 1280) 

handle_base, pixels, FV_SIZE = model_selection

IMAGE_SIZE = (pixels, pixels)

MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,))

feature_extractor.trainable = True

In [3]:
splits = ['train[:10%]', 'train[90%:95%]', 'train[95%:]']
splits, info = tfds.load('cats_vs_dogs', split = splits, with_info=True)
(train_examples, validation_examples, test_examples) = splits

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete2W1NKV/cats_vs_dogs-train.tfrecord


Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [4]:
num_examples = 2500
num_classes = 2

In [5]:
 #This will turn the 3 sets into batches
# so we can train
# This code should not be changed

def format_image(features):
  image = features['image']
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, features['label']
    
BATCH_SIZE =  32

train_batches = train_examples.shuffle(num_examples).map(format_image).batch(BATCH_SIZE)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE)
test_batches = test_examples.map(format_image).batch(BATCH_SIZE)

In [6]:
# The new model will take the features from the mobilenet via transfer learning
# And add a new dense layer at the bottom, with 2 classes -- for cats and dogs

model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(2, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,226,434
Non-trainable params: 34,112
_________________________________________________________________


In [7]:
# Compile the model with adam optimizer and sparse categorical crossentropy, 
# and track the accuracy metric
    
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [8]:
 #Train it for a number of epochs. You should not need many
# Train on the train_Batches, and validation on the validation_batches
EPOCHS = 10

history = model.fit(train_batches, epochs=EPOCHS, validation_data=validation_batches)

Epoch 1/10
73/73 [==============================] - 56s 250ms/step - loss: 0.4055 - acc: 0.8790 - val_loss: 0.3844 - val_acc: 0.9304
Epoch 2/10
73/73 [==============================] - 18s 227ms/step - loss: 0.1763 - acc: 0.9689 - val_loss: 0.6192 - val_acc: 0.9011
Epoch 3/10
73/73 [==============================] - 18s 229ms/step - loss: 0.1889 - acc: 0.9645 - val_loss: 0.2491 - val_acc: 0.9604
Epoch 4/10
73/73 [==============================] - 18s 232ms/step - loss: 0.1332 - acc: 0.9855 - val_loss: 0.2501 - val_acc: 0.9579
Epoch 5/10
73/73 [==============================] - 18s 233ms/step - loss: 0.1236 - acc: 0.9890 - val_loss: 0.2870 - val_acc: 0.9450
Epoch 6/10
73/73 [==============================] - 18s 232ms/step - loss: 0.1493 - acc: 0.9812 - val_loss: 0.4966 - val_acc: 0.9080
Epoch 7/10
73/73 [==============================] - 18s 230ms/step - loss: 0.1235 - acc: 0.9839 - val_loss: 0.3513 - val_acc: 0.9398
Epoch 8/10
73/73 [==============================] - 18s 231ms/step - 

In [9]:
# Evaluate the model on the test batches
eval_results = model.evaluate(test_batches)

# And print the results. You should have >93% accuracy
for metric, value in zip(model.metrics_names, eval_results):
    print(metric + ': {:.4}'.format(value))

37/37 [==============================] - 2s 47ms/step - loss: 0.4538 - acc: 0.9286
loss: 0.4538
acc: 0.9286
